- sections : '정치', '경제', '사회'
- sections = ["pol", "eco","soc"]
- section_ids = ["100", "101","102"]

In [1]:
import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime
import warnings
warnings.filterwarnings('ignore')
import re
import pandas as pd
import time
from datetime import datetime

In [2]:
news_df = pd.DataFrame(
    data=None
    , index=None
    , columns=['date','title','link''content']
)

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

중복 제거 후 행 개수:  0


In [3]:
# ConnectionError방지
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    res = requests.get(url, headers = headers, verify=False)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup

In [4]:
# 뉴스의 기본 정보 가져오기
def get_top3_news_info():
    news_urls =[]
    
    # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101"
    
    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)
  
    # 해당 분야 상위 뉴스 3개 가져오기

    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=3)
    for li in lis3:
        news_url = li.a.attrs.get('href')
        news_urls.append(news_url)
        
    return news_urls

In [5]:
# 상위 3개 뉴스 크롤링
def F_crawling(news_urls) :
    news_titles = []
    news_contents =[]
    news_dates = []

    for url in news_urls:
        news_html = get_soup_obj(url)

        # 뉴스 제목 가져오기
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title == None:
            title = news_html.select_one("#content > div.end_ct > div > h2")

        # 뉴스 본문 가져오기
        content = news_html.select("div#dic_area")
        if content == []:
            content = news_html.select("#articeBody")
            
        # 기사 텍스트만 가져오기
        # list합치기
        content = ''.join(str(content))

        # html태그제거 및 텍스트 다듬기
        pattern1 = '<[^>]*>'
        title = re.sub(pattern=pattern1, repl='', string=str(title))
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')

        news_titles.append(title)
        news_contents.append(content)

        html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
        news_date = html_date.attrs['data-date-time']
        news_dates.append(news_date)
    
    a = pd.DataFrame({'date':news_dates,'title':news_titles,'link':news_urls,'content':news_contents})

    #중복 행 지우기
    a = a.drop_duplicates(keep='first',ignore_index=True)

    return a

In [6]:
# 처음 크롤링 시작
while True :
    news_urls = get_top3_news_info()
    a = F_crawling(news_urls)
    news_df = pd.concat([news_df, a], ignore_index=True)
    news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
    print(news_df.tail())

    time.sleep(10)

                  date                             title linkcontent  \
0  2023-07-06 10:54:11  동국제강그룹, 창립 69주년…"여러분이 첫 선배" 의미는?         NaN   
1  2023-07-06 10:54:01   [프로필] 공정위 부위원장에 '조사통' 조홍선 조사관리관         NaN   
2  2023-07-06 10:54:01        군사용 드론 ‘하드 킬’  시험 이렇게 이뤄진다         NaN   

                                                link  \
0  https://n.news.naver.com/mnews/article/003/001...   
1  https://n.news.naver.com/mnews/article/277/000...   
2  https://n.news.naver.com/mnews/article/023/000...   

                                             content  
0  [\n\n\n\n\n[서울=뉴시스]6일 서울 수하동 페럼타워에서 열린 동국홀딩스 창...  
1  [\n\n\n\n\n윤석열 대통령이 공정거래위원회 부위원장으로 조홍선 현 공정거래위...  
2  [\n\t\t\t\t\t\t\t\t\t\t현대차그룹 계열사인 현대위아가 국내 최초로...  
                  date                                   title linkcontent  \
1  2023-07-06 10:54:01         [프로필] 공정위 부위원장에 '조사통' 조홍선 조사관리관         NaN   
2  2023-07-06 10:54:01              군사용 드론 ‘하드 킬’  시험 이렇게 이뤄진다         NaN   
3  2023-07-06 10:55:03  현대모비스·LG

KeyboardInterrupt: 

In [8]:
news_df.tail(10)

,date,title,linkcontent,link,content
1,2023-07-06 10:54:01,[프로필] 공정위 부위원장에 '조사통' 조홍선 조사관리관,NaN,https://n.news.naver.com/mnews/article/277/000...,[\n\n\n\n\n윤석열 대통령이 공정거래위원회 부위원장으로 조홍선 현 공정거래위...
2,2023-07-06 10:54:01,군사용 드론 ‘하드 킬’ 시험 이렇게 이뤄진다,NaN,https://n.news.naver.com/mnews/article/023/000...,[\n\t\t\t\t\t\t\t\t\t\t현대차그룹 계열사인 현대위아가 국내 최초로...
3,2023-07-06 10:55:03,"현대모비스·LG에너지솔루션 인니 합작공장, 아시아 ‘올해의 딜’ 선정",NaN,https://n.news.naver.com/mnews/article/021/000...,"[\n\n\n\n\n현대모비스 CI. 현대모비스 제공 ""아시아 지역 전기차 전환에..."
4,2023-07-06 10:55:03,[프로필] 고광효 신임 관세청장…尹 정부 첫 세제개편 주도,NaN,https://n.news.naver.com/mnews/article/016/000...,[\n\n\n\n\n고광효 신임 관세청장[헤럴드경제=배문숙 기자]고광효 신임 관세청...
5,2023-07-06 10:55:01,"현대위아, ‘대 드론 체계’ 국내 최초 하드 킬 사격 시험 성공",NaN,https://n.news.naver.com/mnews/article/021/000...,[\n\n\n\n\n현대위아 현대위아가 지난 5일 충북 소재 사격훈련장에서 ADS ...
